# Day09

In [163]:
from time import perf_counter_ns
from time import sleep
from collections import deque

with open('input.txt', 'r') as f:
    data = f.read()

#print(data)
#print("00...111...2...333.44.5555.6666.777.888899\n0099.111...2...333.44.5555.6666.777.8888..\n0099.1117772...333.44.5555.6666.....8888..\n0099.111777244.333....5555.6666.....8888..\n00992111777.44.333....5555.6666.....8888..")

## Part 1

In [ ]:
start_p1 = perf_counter_ns()

blocks = deque()
for i in range(0, len(data), 2):
    blocks.extend(str(i//2) for _ in range(int(data[i])))
    blocks.extend('.' for _ in range(int(data[i+1]) if i+1 < len(data) else 0))    

nogaps = []
dots=0

while blocks:
    block = blocks.popleft()
    
    if block != '.':
        nogaps.append(block)
    else:
        dots += 1
        while True:
            right = blocks.pop()
            if right == '.':
                dots += 1
            else:
                nogaps.append(right)
                break

nogaps = nogaps + ['.' for _ in range(dots)]      

checksum = 0
for i in range(len(nogaps)):
    if nogaps[i] == '.':
        break
    checksum += int(nogaps[i]) * i
    
print("Part 1:", checksum)
time_p1 = round((perf_counter_ns() - start_p1),2) / 1e6
print("Time taken:", time_p1, "ms")

Part 1: 6301895872542
Time taken: 43.780818 ms


## Part 2

In [166]:
start_p2 = perf_counter_ns()

blocks = []
for i in range(0, len(data), 2):
    blocks.extend(str(i//2) for _ in range(int(data[i])))
    blocks.extend('.' for _ in range(int(data[i+1]) if i+1 < len(data) else 0))

def group_consecutive(lst):
    if not lst:
        return []
        
    result = []
    current_group = [lst[0]]
    
    for item in lst[1:]:
        if item == current_group[0]:
            current_group.append(item)
        else:
            result.append(current_group)
            current_group = [item]
            
    result.append(current_group)
    return result

def swap_elements(lst, index1, index2):
    if index1 >= len(lst) or index2 >= len(lst) or index1 < 0 or index2 < 0:
        return lst
    lst[index1], lst[index2] = lst[index2], lst[index1]
    return lst

def split_sublist(lst, index, diff):
    if index >= len(lst):
        return lst
        
    target = lst[index]
    if len(target) <= diff:
        return lst
        
    first_part = target[:diff]
    second_part = target[diff:]
    
    lst[index:index+1] = [second_part, first_part]
    
    return lst

def join_dots(lst):
    i = 0
    while i < len(lst) - 1:
        if lst[i][0] == '.' and lst[i+1][0] == '.':
            lst[i] = lst[i] + lst[i+1]
            lst.pop(i+1)
        else:
            i += 1
    return lst

def swap_groups(lst, index1, index2):
    len1 = len(lst[index1])
    len2 = len(lst[index2])
    
    if len1 < len2:
        return lst
    if len1 == len2:
        return swap_elements(lst, index1, index2), 0
    
    diff = len1 - len2
    lst = split_sublist(lst, index1, diff)
    lst = swap_elements(lst, index1, index2+1)
    lst = join_dots(lst)
    return lst, 1

separated = group_consecutive(blocks)

size = len(separated)
id = int(separated[-1][0])
i=len(separated)-1
while id>0:
    if separated[i][0] == '.':
        i-=1
        continue
    
    for j in range(i):
        if separated[j][0] == '.' and len(separated[j]) >= len(separated[i]):
            separated,add = swap_groups(separated, j, i)
            i+=add
            break
        
    separated = join_dots(separated)
    id-=1
    i-=1

final=[]
checksum = 0
for i in range(len(separated)):
    final.extend(separated[i])

for i in range(len(final)):
    if final[i] == '.':
        continue
    checksum += int(final[i]) * i

print("Part 2:", checksum)
time_p2 = round((perf_counter_ns() - start_p2), 2) / 1e6
print("Time taken:", time_p2, "ms")

Part 2: 6323761685944
Time taken: 16141.458343 ms
